**Описание данных:**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Открытие файла с данными и изучение общей информации

In [126]:
import pandas as pd
borrowers = pd.read_csv('/datasets/data.csv')
borrowers.info() #смотрим общую информацию по таблице

from IPython.display import display

display(borrowers.head())
#смотрим какие ошибки можно сразу увидеть в данных
#display(borrowers.tail()) #смотрим какие ошибки можно сразу увидеть в данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод** 
1. В таблице 21525 строк, 12 столбцов. Пропущены данные в столбцах "общий трудовой стаж в днях", "ежемесячный доход". Пропущено по 10%, одинаковое количество.

2. В столбце "общий трудовой стаж в днях" встречаются отрицательные значения, также данные указаны как вещественные.

3. Данные в столбце "уровень образования клиента" указаны и большими и маленькими буквами.

4. в столбце "цель получения кредита" одна и та же цель  указана по-разному 


## Шаг 2. Предобработка данных

### Обработка пропусков

In [127]:
#1. Сочитаем количество пропущенных значений

print(borrowers.isnull().sum()) #В данных ежемес.дохода и труд.стажа одинаковое количество пропущенных значений. Пропущено по 10%, 
#достаточно много для того, чтобы их просто исключить,поэтому будем заполнять пробелы соответствующими средними значениями. 


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [128]:
#2. Посмотрим как распределяются пропущенные данные ежем.дохода по типам занятости

bor1 = borrowers[borrowers['total_income'].isna()]
print(bor1['income_type'].value_counts()) #Вероятно клиенты не указывают информацию по доходу из-за его отсутствия (пенсионеры,
# студенты), из-за нежелания его указывать (госслужащие)

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [129]:
#3. Смотрим как распределяются пропущенные данные труд.стажа по типам занятости

bor2 = borrowers[borrowers['days_employed'].isna()]
print(bor2['income_type'].value_counts()) 

#Т.к. количество пропущенных значений по доходу и стажу и распределение по видам занятости одинаковое,
#есть вероятность ошибки в выгруженных данных

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [130]:
#4. Для устранения пропусков в столбце труд.стажа и ежем.дохода вычислим средний труд.стаж и средний ежем.доход и заменим ими
#пропущенные значения в соответствующих столбцах:
 
borrowers['days_employed'] = borrowers['days_employed'].abs() #меняем отрицательные значения в столбце труд.стажа на положительные
days_employed_avg = borrowers['days_employed'].median() #находим средний туд.стаж
total_income_avg = borrowers['total_income'].median() #находим средний ежем.доход
borrowers = borrowers.fillna({'days_employed' : days_employed_avg, 'total_income' : total_income_avg})
borrowers.info() #проверяем все ли пропущенные данные в столбце трудового стажа заполнили

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [131]:
#6.Поищем возможные ошибки в данных о количестве детей:
print(borrowers['children'].min(), borrowers['children'].max())
#Выявили отриц.число в данных и очень большое максимальное

-1 20


In [132]:
#6.1 меняем отрицательные значения в столбце на положительные
borrowers['children'] = borrowers['children'].abs()

In [133]:
#6.2 Т.к.20 детей - большая редкость, то исходим из предположения, что в данных опечатка: к цифре "2" приклеился"0". Заменим 20 на 2.
borrowers['children'] = borrowers['children'].replace(20,2)
print(borrowers['children'].value_counts())  #проверяем

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [134]:
#7.Поищем возможные ошибки в данных о задолженностях:
print(borrowers['debt'].value_counts()) #некорректные данных нет

0    19784
1     1741
Name: debt, dtype: int64


In [135]:
#8. Посмотрим на данные по возрасту клиентов:
print(borrowers['dob_years'].min(), borrowers['dob_years'].max())

0 75


In [136]:
#8.1 Груднички определенно еще не интересуются кредитами, посмотрим на % таких строчек в общей выборке:
print(len(borrowers[borrowers['dob_years']==0])/len(borrowers))  #Менее 0,5% не играет существенной роли, но для красоты заменим на средний возраст
dob_years_avg = borrowers['dob_years'].mean()
borrowers['dob_years'] = borrowers['dob_years'].replace(0,dob_years_avg)
print(borrowers['dob_years'].min())  #проверка

0.004692218350754936
19.0


**Вывод** Нашли пропущенные значения в данных по ежемесячному доходу и трудовому стажу, заменили их на соответствующие средние значения. Пропушенных значений больше нет. Также наши некорректные данные в количестве детей, и в возрасте клиентов.

### Замена типа данных

In [137]:
#Данные по стажу не могут быть вещественными числами, изменим тип данных на целочисленный:
borrowers['days_employed'] = borrowers['days_employed'].astype('int') 
borrowers.info() #проверяем результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Данные в столбце "трудовой стаж в днях" не могут быть вещественными, изменили тип данных на integer.

### Обработка дубликатов

In [138]:
#1.1 Посчитаем количество грубых дубликатов:
print(borrowers.duplicated().sum())

54


In [139]:
#1.2 Очищаем таблицу от грубых дубликатов:
borrowers = borrowers.drop_duplicates().reset_index(drop=True) 
print(borrowers.duplicated().sum()) #делаем проверку

0


In [140]:
#2.1 Проверим наличие других дубликатов, посмотрим уникальные значения по столбцу education
print(borrowers['education'].value_counts()) #данные в столбце отличаются регистром

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [141]:
#2.2 Приведем данные в данном столбце к нижнему регистру
borrowers['education'] = borrowers['education'].str.lower()
print(borrowers['education'].value_counts()) #делаем проверку

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [142]:
#2.3 Проверим не появились ли новые дубликаты 
print(borrowers.duplicated().sum())

17


In [143]:
#2.4 Очищаем таблицу от новых дубликатов:
borrowers = borrowers.drop_duplicates().reset_index(drop=True) 
print(borrowers.duplicated().sum()) #делаем проверку

0


In [144]:
#3. Аналогично, проверим уникальные значения в столбцах семейное положение и тип занятости
print(borrowers['family_status'].value_counts()) #в данных нет дубликатов 

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [145]:
print(borrowers['income_type'].value_counts())  #в данных нет дубликатов 

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


**Вывод** В данных выявили дубликаты и наличие текста с разным регистром. Привели все к нижнему регистру и удалили дубликаты.

### Лемматизация

In [146]:
#1. Посмотрим для каких целей берут кредит
print(borrowers['purpose'].value_counts()) # Как видим, одна и та же цель записана по-разному, приведем к единообразию

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [147]:
#2. Приводим цели к единообразию
from pymystem3 import Mystem
m = Mystem()

def purpose_rename(row,word):
    for cell in row:
        lemmas = m.lemmatize(cell)
        if word in lemmas:
            row = row.replace(cell, word)
               
    return row

borrowers['purpose'] = purpose_rename(borrowers['purpose'], 'свадьба')
borrowers['purpose'] = purpose_rename(borrowers['purpose'], 'недвижимость')
borrowers['purpose'] = purpose_rename(borrowers['purpose'], 'жилье')
borrowers['purpose'] = purpose_rename(borrowers['purpose'], 'автомобиль')
borrowers['purpose'] = purpose_rename(borrowers['purpose'], 'образование')
    
print(borrowers['purpose'].value_counts())  #проверяем

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64


In [148]:
# Т.к. жилье и недвижимость из одной группы, переименуем 
borrowers['purpose'] = borrowers['purpose'].replace('жилье','недвижимость')
print(borrowers['purpose'].value_counts()) #проверяем

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64


**Вывод** Клиенты указали цели по-разному, но их можно распределить по смысловым группам "недвижимость", "автомобиль", "образование", "свадьба".

### Категоризация данных

In [149]:
# Создадим словарь:
df = pd.DataFrame({'debt_id':['without_debt','with_debt'], 'debt': [0,1]}) #создаем вспомогательную таблицу/словарь
debt_sl = df[['debt_id','debt']]
debt_sl['debt'] = borrowers['debt'].drop_duplicates().reset_index(drop=True)
print(debt_sl)


        debt_id  debt
0  without_debt     0
1     with_debt     1


In [164]:
#1.1 Для ответа на первый вопрос создадим дополнительную таблицу:
table1 = borrowers[['children','debt']]


In [165]:
#1.2 Для построения сводной таблицы объединим словарь и дополнительную таблицу:
children_ver_debt = table1.merge(debt_sl, on='debt',how='left')
#display(children_ver_debt.tail(10))  #проверка

In [166]:
#2. Для ответа на второй вопрос создадим дополнительную таблицу:
table2 =  borrowers[['family_status', 'debt']]

In [167]:
#2.1 Для построения сводной таблицы объединим словарь и дополнительную таблицу:
family_ver_debt = table2.merge(debt_sl, on='debt',how='left')
#display(family_ver_debt.tail(10))  #проверка

In [168]:
#3. Для ответа на третий вопрос, распределим данные по категориям относительно уровня дохода.
#3.1 Сосчитаем средний уровень дохода:
total_income_avg = borrowers['total_income'].median()
print(total_income_avg)
print(borrowers['total_income'].min(), borrowers['total_income'].max())

145017.93753253992
20667.26379327158 2265604.028722744


In [169]:
#3.2 Возьмем две группы данных-до среднего, и после среднего. В каждой из этих групп найдем среднее:
median1 = borrowers.loc[borrowers['total_income'] < total_income_avg, 'total_income'].median()
median2 = borrowers.loc[borrowers['total_income'] >= total_income_avg, 'total_income'].median()
print(median1,median2)

103028.28843296492 187855.36677355366


In [170]:
#3.3 Теперь разобьем на категории по уровню дохода: 
# все, что меньше или равно median1 - "низкий", больше median1, но меньше 
# все, что больше median1, но меньше или равно median2 - "средний"
# все, что выше median2 - "высокий"
def income_group(income):
    if income <= median1:
        return 'низкий'
    if income <= median2:
        return 'средний'
    return 'высокий'

borrowers['income_group'] = borrowers['total_income'].apply(income_group)
#display(borrowers.head()) #проверка

In [171]:
#3.4 Cоздадим дополнительную таблицу:
table3 =  borrowers[['income_group', 'debt']]

In [172]:
#3.5 Для построения сводной таблицы объединим словарь и дополнительную таблицу:
income_ver_debt = table3.merge(debt_sl, on='debt',how='left')
#display(income_ver_debt.tail(10))  #проверка

In [173]:
#4.1 Для ответа на четвертый вопрос создадим дополнительную таблицу:
table4 =  borrowers[['purpose', 'debt']]

In [174]:
#4.2 Для построения сводной таблицы объединим словарь и дополнительную таблицу:
purpose_ver_debt = table4.merge(debt_sl, on='debt',how='left')
#display(purpose_ver_debt.tail(10))  #проверка

**Вывод** Для дальнейшей работы категоризировали данные.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [175]:
#1.1. Создадим сводную таблицу:
children_pivot = children_ver_debt.pivot_table(index='children', columns='debt_id', values='debt', aggfunc='count')
display(children_pivot)


debt_id,with_debt,without_debt
children,,
0,1063.0,13028.0
1,445.0,4410.0
2,202.0,1926.0
3,27.0,303.0
4,4.0,37.0
5,NaN,9.0


In [176]:
#1.2. Т.к.нет клиентов с 5 детьми, не вернувшими кредит в срок, поставим в соответствующей ячейке 0
children_pivot['with_debt'] = children_pivot['with_debt'].fillna(0)
#display(children_pivot)

In [177]:
#1.3 Добавим новый столбец, в котором укажем отношение клиентов, не вернувших долг в срок, к обшему количеству клиентов в каждой группе:
children_pivot['debt_probability'] = children_pivot['with_debt'] / (children_pivot['with_debt'] + children_pivot['without_debt']) 
  
display(children_pivot.sort_values(by='debt_probability', ascending=False))

debt_id,with_debt,without_debt,debt_probability
children,,,
4,4.0,37.0,0.097561
2,202.0,1926.0,0.094925
1,445.0,4410.0,0.091658
3,27.0,303.0,0.081818
0,1063.0,13028.0,0.075438
5,0.0,9.0,0.000000


In [178]:
#1.4 Добавим дополнительный столбец, чтобы оценить долю каждой группы в общей выборке:
all = len(borrowers)
children_pivot['percent_in_all'] = (children_pivot['with_debt'] + children_pivot['without_debt']) / all
display(children_pivot)

#Учитывая маленький процент многодетных семей в общей выборке, можем смело говорить о том, что с ростом числа детей вероятность невозврата кредита в срок увеличивается.

debt_id,with_debt,without_debt,debt_probability,percent_in_all
children,,,,
0,1063.0,13028.0,0.075438,0.656801
1,445.0,4410.0,0.091658,0.226298
2,202.0,1926.0,0.094925,0.099189
3,27.0,303.0,0.081818,0.015382
4,4.0,37.0,0.097561,0.001911
5,0.0,9.0,0.000000,0.000420


**Вывод** Чем меньше у клиентов детей, тем чаще они обращаются за кредитом. А с увеличением числа детей вероятность невозврата кредита в срок увеличивается. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [179]:
#2.1 Создадим сводную таблицу:
family_pivot = family_ver_debt.pivot_table(index='family_status', columns='debt_id', values='debt', aggfunc='count')
display(family_pivot)

debt_id,with_debt,without_debt
family_status,,
Не женат / не замужем,274,2536
в разводе,85,1110
вдовец / вдова,63,896
гражданский брак,388,3763
женат / замужем,931,11408


In [180]:
#2.2 Добавим новый столбец, в котором укажем отношение клиентов, не вернувших долг в срок, к обшему количеству клиентов в каждой группе:
family_pivot['debt_probability'] = family_pivot['with_debt'] / (family_pivot['with_debt'] + family_pivot['without_debt']) 
  
display(family_pivot.sort_values(by='debt_probability', ascending=False))

debt_id,with_debt,without_debt,debt_probability
family_status,,,
Не женат / не замужем,274,2536,0.097509
гражданский брак,388,3763,0.093471
женат / замужем,931,11408,0.075452
в разводе,85,1110,0.071130
вдовец / вдова,63,896,0.065693


In [181]:
#2.3 Добавим дополнительный столбец, чтобы оценить долю каждой группы в общей выборке:
family_pivot['percent_in_all'] = (family_pivot['with_debt'] + family_pivot['without_debt']) / all
display(family_pivot)

debt_id,with_debt,without_debt,debt_probability,percent_in_all
family_status,,,,
Не женат / не замужем,274,2536,0.097509,0.130978
в разводе,85,1110,0.071130,0.055701
вдовец / вдова,63,896,0.065693,0.044700
гражданский брак,388,3763,0.093471,0.193484
женат / замужем,931,11408,0.075452,0.575138


**Вывод** Вероятность невозврата кредита в срок самая высокая у одиноких людей. Самые ответственные люди - те, кто имеет или имел штамп в паспорте.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [182]:
#3.1 Создадим сводную таблицу:
income_pivot = income_ver_debt.pivot_table(index='income_group', columns='debt_id', values='debt', aggfunc='count')
display(income_pivot)

debt_id,with_debt,without_debt
income_group,,
высокий,433,5456
низкий,383,4455
средний,925,9802


In [183]:
#3.2 Добавим новый столбец, в котором укажем отношение клиентов, не вернувших долг в срок, к обшему количеству клиентов в каждой группе:
income_pivot['debt_probability'] = income_pivot['with_debt'] / (income_pivot['with_debt'] + income_pivot['without_debt']) 
  
display(income_pivot.sort_values(by='debt_probability', ascending=False))

debt_id,with_debt,without_debt,debt_probability
income_group,,,
средний,925,9802,0.086231
низкий,383,4455,0.079165
высокий,433,5456,0.073527


In [184]:
#3.3 Добавим дополнительный столбец, чтобы оценить долю каждой группы в общей выборке:
income_pivot['percent_in_all'] = (income_pivot['with_debt'] + income_pivot['without_debt']) / all
display(income_pivot)

debt_id,with_debt,without_debt,debt_probability,percent_in_all
income_group,,,,
высокий,433,5456,0.073527,0.274494
низкий,383,4455,0.079165,0.225506
средний,925,9802,0.086231,0.500000


**Вывод** Самый высокий процент возврата кредита в срок - у клиентов с высоким уровнем дохода, а вот самый низкий у людей со средним уровнем.

- Как разные цели кредита влияют на его возврат в срок?

In [185]:
#4.1 Создадим сводную таблицу:
purpose_pivot = purpose_ver_debt.pivot_table(index='purpose', columns='debt_id', values='debt', aggfunc='count')
display(purpose_pivot)

debt_id,with_debt,without_debt
purpose,,
автомобиль,403,3903
недвижимость,782,10029
образование,370,3643
свадьба,186,2138


In [186]:
#4.2 Добавим новый столбец, в котором укажем отношение клиентов, не вернувших долг в срок, к обшему количеству клиентов в каждой группе:
purpose_pivot['debt_probability'] = purpose_pivot['with_debt'] / (purpose_pivot['with_debt'] + purpose_pivot['without_debt']) 
  
display(purpose_pivot.sort_values(by='debt_probability', ascending=False))

debt_id,with_debt,without_debt,debt_probability
purpose,,,
автомобиль,403,3903,0.093590
образование,370,3643,0.092200
свадьба,186,2138,0.080034
недвижимость,782,10029,0.072334


In [187]:
#4.3 Добавим дополнительный столбец, чтобы оценить долю каждой группы в общей выборке:
purpose_pivot['percent_in_all'] = (purpose_pivot['with_debt'] + purpose_pivot['without_debt']) / all
display(purpose_pivot)

debt_id,with_debt,without_debt,debt_probability,percent_in_all
purpose,,,,
автомобиль,403,3903,0.093590,0.200708
недвижимость,782,10029,0.072334,0.503915
образование,370,3643,0.092200,0.187051
свадьба,186,2138,0.080034,0.108325


**Вывод** Самый популярный кредит - кредит на недвижимость и процент возврата его в срок - самый высокий, а вот кредиты на автомобиль возвращают в срок реже всего. 

## Шаг 4. Общий вывод

Самые безответственные клиенты - одинокие люди со средним уровнем дохода, с детьми, которые планируют купить авто.
А вот идеальные клиенты - люди с высоким уровнем дохода, без детей, штампом в паспорте, цель которых - кредит на недвижимость.